In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.float_format = '{:,}'.format


In [3]:
df = pd.read_excel('BC LCG 2017 CN (1).xlsx', sheet_name = 2, skiprows = 1, converters={'Departamento - Clave': str, 'Número de Vendedor':str, 'Número de cliente':str, 'Familia - Clave':str})
claves = pd.read_excel('BC LCG 2017 CN (1).xlsb', sheet_name = 1, skiprows = 1, dtype=str)


In [4]:
departamento_clave = claves[['Departamento - Clave','Departamento']]
vendedores_clave = claves[['No. Vendedor','Nombre', 'Apellido']]
vendedores_clave.dropna(axis = 0, inplace = True)
vendedores_clave['Nombre Completo'] = vendedores_clave.Nombre.str.cat(vendedores_clave.Apellido, sep = " ") 
vendedores_clave.rename({'No. Vendedor':'Número de Vendedor'}, axis = 1, inplace = True)

In [5]:
#Limpieza de Base de Datos

df.drop('Unnamed: 2', axis = 1, inplace = True)
df['Nom_Completo_Vendedor'] = pd.merge(df, vendedores_clave[['Nombre Completo','Número de Vendedor']], on = 'Número de Vendedor', how = 'left')['Nombre Completo']
df = pd.merge(df, departamento_clave, on = 'Departamento', how = 'left')
df['Departamento - Clave_x'].fillna(df['Departamento - Clave_y'], inplace = True)
df.drop(['Departamento - Clave_y'], axis = 1, inplace = True)
df.rename({'Departamento - Clave_x':'Departamento - Clave'}, axis = 1, inplace = True)
df['Departamento'] = pd.merge(df, departamento_clave.rename({'Departamento':'Departamento1'}, axis = 1), on = 'Departamento - Clave', how = 'left')['Departamento1']
df.dropna(axis = 0, how = 'all', inplace = True)

#1 USD = 7.5Q
df['Ventas Netas USD'] = round(df['Ventas Netas (Q)']/7.5,2)

#Formato de Fecha
df['Fecha'] = pd.to_datetime(df['Fecha'])

df = df.round(2)


In [6]:
#Cual es el monto de ventas por cada clasificacion (A, B, C)

pivot1 = df.pivot_table(index = 'Departamento', values = 'Ventas Netas USD' ,aggfunc = 'sum')
pivot1['% Ventas Netas'] = pivot1['Ventas Netas USD']/pivot1.sum()[0] * 100
pivot1.sort_values(by = '% Ventas Netas', ascending = False, inplace = True)
pivot1['% Ventas Netas Acumuladas'] = pivot1['% Ventas Netas'].cumsum()

def clasificacion(row):
    
    if row['% Ventas Netas Acumuladas'] <= 80:
        return 'A'
    elif 80 < row['% Ventas Netas Acumuladas'] <= 95:
        return 'B'
    else:
        return 'C'
    
pivot1['Clasificacion'] = pivot1.apply(clasificacion, axis = 1)
respuesta1 = pivot1.groupby('Clasificacion').sum()[['Ventas Netas USD']]

for i in range(len(respuesta1.index)):
    
    print(f"Cantidad de Ventas para {respuesta1.index[i]}:{round(respuesta1.iloc[i][0],2):,}")


Cantidad de Ventas para A:184,530,954.87
Cantidad de Ventas para B:35,356,586.6
Cantidad de Ventas para C:14,864,808.49


In [7]:
#Cual es el numero de departamentos por cada clasificacion (A, B, C)

respuesta2 = pivot1.groupby('Clasificacion').count()[['Ventas Netas USD']]

for i in range(len(respuesta2.index)):
    
    print(f"Cantidad de departamentos para {respuesta2.index[i]}:{respuesta2.iloc[i][0]}")


Cantidad de departamentos para A:18
Cantidad de departamentos para B:7
Cantidad de departamentos para C:7


In [8]:
#Realizar un ABC de clientes e identificar cuales son los 5 clientes que mas compran
pivot2 = df.pivot_table(index = 'Número de cliente', values = 'Ventas Netas USD' ,aggfunc = 'sum')
pivot2['% Ventas Netas'] = (pivot2['Ventas Netas USD']/pivot2.sum()[0]) * 100
pivot2.sort_values(by = '% Ventas Netas', ascending = False, inplace = True)
pivot2['% Ventas Netas Acumuladas'] = pivot2['% Ventas Netas'].cumsum()
pivot2['Clasificacion'] = pivot2.apply(clasificacion, axis = 1)

pivot2 = pivot2.round(2)

print(f"Los Clientes que mas compran son:{list(pivot2.head().index)}")
for i in range(len(pivot2.head())):
    print(f"cliente {pivot2.index[i]} con {pivot2.iloc[i][0]:,}")
pivot2.head()


Los Clientes que mas compran son:['47', '14', '59', '68', '74']
cliente 47 con 20,530,094.3
cliente 14 con 19,963,858.28
cliente 59 con 19,198,607.43
cliente 68 con 18,611,336.37
cliente 74 con 16,949,299.53


,Ventas Netas USD,% Ventas Netas,% Ventas Netas Acumuladas,Clasificacion
Número de cliente,,,,
47,"20,530,094.3",8.75,8.75,A
14,"19,963,858.28",8.5,17.25,A
59,"19,198,607.43",8.18,25.43,A
68,"18,611,336.37",7.93,33.36,A
74,"16,949,299.53",7.22,40.58,A


In [9]:
#Identificar cual es el vendedor que le vende a la mayoria de clientes

pivot3 = df.pivot_table(index = ['Número de Vendedor','Nom_Completo_Vendedor'], values = 'Ventas Netas USD' ,aggfunc = 'sum')
pivot3['% Ventas Netas'] = (pivot3['Ventas Netas USD']/pivot3.sum()[0]) * 100
pivot3.sort_values(by = '% Ventas Netas', ascending = False, inplace = True)
pivot3['% Ventas Netas Acumuladas'] = pivot3['% Ventas Netas'].cumsum()
pivot3['Clasificacion'] = pivot3.apply(clasificacion, axis = 1)

print("La persona que mas vende es Juan Garza, es decir el vendedor numero 1")
pivot3


La persona que mas vende es Juan Garza, es decir el vendedor numero 1


,,Ventas Netas USD,% Ventas Netas,% Ventas Netas Acumuladas,Clasificacion
Número de Vendedor,Nom_Completo_Vendedor,,,,
1,Juan Garza,"126,053,855.53",53.69652553061923,53.69652553061923,A
8,Ernesto Pineda,"27,277,862.36",11.619846346436123,65.31637187705536,A
5,Gonzalo Aguilar,"15,621,483.8",6.654452576369005,71.97082445342437,A
3,Sofia Gonzalez,"15,238,778.75",6.491427562960103,78.46225201638447,A
7,Carlos Vela,"14,234,057.75",6.063435681229762,84.52568769761423,B
6,Paulina Casanova,"14,146,193.76",6.026007306171974,90.55169500378621,B
4,Pedro Arroyo,"13,928,986.03",5.93348097787877,96.48517598166498,C
2,Michelle Perez,"8,251,131.98",3.5148240183350374,100.00000000000001,C


In [10]:
#Cual es el mes con mas ventas para el año 2015 y 2016?

pivot4 = df.pivot_table(index = ['Fecha'], values = 'Ventas Netas USD' ,aggfunc = 'sum')
max_2015 = str(pivot4.loc[(pivot4.index > '2014-12-31') & (pivot4.index < '2016-01-01')].sort_values(by = 'Ventas Netas USD', ascending = False).idxmax()[0])[:-8]
max_2016 = str(pivot4.loc[(pivot4.index > '2015-12-31') & (pivot4.index < '2017-01-01')].sort_values(by = 'Ventas Netas USD', ascending = False).idxmax()[0])[:-8]

maximos = [max_2015, max_2016]
años = [2015, 2016]

for i,j in zip(maximos,años):
    
    print(f"El mes con mas ventas para el año {j} es {i} con ${pivot4.loc[i][0]:,}")

El mes con mas ventas para el año 2015 es 2015-12-01  con $23,388,850.84
El mes con mas ventas para el año 2016 es 2016-11-01  con $16,960,601.28


In [11]:
pivot4 = df.pivot_table(index = ['Fecha'], values = 'Ventas Netas USD' ,aggfunc = 'sum')
max_2015 = str(pivot4.loc[(pivot4.index > '2014-12-31') & (pivot4.index < '2016-01-01')].sort_values(by = 'Ventas Netas USD', ascending = False).idxmin()[0])[:-8]
max_2016 = str(pivot4.loc[(pivot4.index > '2015-12-31') & (pivot4.index < '2017-01-01')].sort_values(by = 'Ventas Netas USD', ascending = False).idxmin()[0])[:-8]

minimos = [max_2015, max_2016]
años = [2015, 2016]

for i,j in zip(minimos,años):
    
    print(f"El mes con menos ventas para el año {j} es {i} con ${pivot4.loc[i][0]:,}")

El mes con menos ventas para el año 2015 es 2015-02-01  con $7,084,395.66
El mes con menos ventas para el año 2016 es 2016-12-01  con $4,404,673.12


In [12]:
#Cual es el porcentaje de crecimiento anual?

ventas_totales_2015 = (pivot4.loc[(pivot4.index > '2014-12-31') & (pivot4.index < '2016-01-01')].sum())
ventas_totales_2016 = (pivot4.loc[(pivot4.index > '2015-12-31') & (pivot4.index < '2017-01-01')].sum())

pct_crecimiento = (ventas_totales_2016-ventas_totales_2015)/ventas_totales_2015

print(f"El porcentaje de crecimiento entre estos dos años es: {pct_crecimiento[0]:%}")

El porcentaje de crecimiento entre estos dos años es: -7.181865%


In [13]:
#Cual es la rentabilidad del mes de septiembre del 2016

pivot5 = df.pivot_table(index = ['Fecha'], values = ['Ventas Netas (Q)', 'Costo'] ,aggfunc = 'sum')
pivot5['Rentabilidad'] = round(pivot5['Ventas Netas (Q)'] - pivot5['Costo'],2 )
pivot5['Pct_Rentabilidad'] = round((pivot5['Ventas Netas (Q)'] - pivot5['Costo'])/pivot5['Ventas Netas (Q)'],2 )

print(f"La rentabilidad para septiembre del 2016 es {(pivot5.loc['2016-09-01']['Rentabilidad'])/7.5:,}, es decir, un {pivot5.loc['2016-09-01']['Pct_Rentabilidad']:%}")

La rentabilidad para septiembre del 2016 es 4,485,834.66, es decir, un 44.000000%


In [14]:
# Cual fue el cliente menos rentable y en que año?

pivot6 = df.pivot_table(index = ['Fecha','Número de cliente'], values = ['Ventas Netas (Q)', 'Costo'] ,aggfunc = 'sum')

In [15]:
#pivot6.sort_values(by = 'Ventas Netas (Q)')
pivot6['Rentabilidad'] = round(pivot6['Ventas Netas (Q)'] - pivot6['Costo'],2 )
pivot6['Rentabilidad'] = pivot6['Rentabilidad']/7.5 
pivot6['Pct_Rentabilidad'] = round((pivot6['Ventas Netas (Q)'] - pivot6['Costo'])/pivot6['Ventas Netas (Q)'],2)

In [16]:
pivot_ord = pivot6.sort_values(by = 'Pct_Rentabilidad')
print(f"El cliente numero {str(pivot_ord.iloc[[0]].index[0][1])} fue el cliente menos rentable, esto fue en {str(pivot_ord.iloc[[0]].index[0][0])[:-9]} con un porcentaje de {pivot_ord.iloc[0]['Pct_Rentabilidad']:%} con una rentabilidad de {pivot_ord.iloc[0]['Rentabilidad']/7.5}")

El cliente numero 10 fue el cliente menos rentable, esto fue en 2015-02-01 con un porcentaje de -70.000000% con una rentabilidad de -12.964444444444444


In [18]:
#Cual fue el trimestre con la rentabilidad mas baja?
pivot7 = pivot5.reset_index()
pivot7['trimestre'] = pd.PeriodIndex(pivot7['Fecha'], freq='Q')
pivot7.drop(['Fecha','Pct_Rentabilidad'],axis = 1, inplace = True)
pivot7_Q = pivot7.groupby('trimestre').sum()
pivot7_Q['Pct_Rentabilidad'] = (pivot7_Q['Ventas Netas (Q)'] - pivot7_Q['Costo'])/pivot7_Q['Ventas Netas (Q)']

In [19]:
pivot7_Q.sort_values(by = 'Pct_Rentabilidad', ascending = True)
pivot7_Q['Pct_Rentabilidad'] = pivot7_Q['Pct_Rentabilidad'].astype(float).map(lambda n: '{:.2%}'.format(n))

In [20]:
pivot7_Q

,Costo,Ventas Netas (Q),Rentabilidad,Pct_Rentabilidad
trimestre,,,,
2015Q1,"109,413,899.16","188,750,972.57999998","79,337,073.42",42.03%
2015Q2,"109,492,726.21000001","185,663,574.76","76,170,848.55",41.03%
2015Q3,"116,444,580.22","202,474,824.45999998","86,030,244.24",42.49%
2015Q4,"192,824,490.66","336,221,120.41999996","143,396,629.76",42.65%
2016Q1,"114,113,366.2","201,380,416.45","87,267,050.25",43.33%
2016Q2,"122,777,770.33","206,323,759.41","83,545,989.08",40.49%
2016Q3,"119,345,277.44","215,003,642.5","95,658,365.06",44.49%
2016Q4,"126,952,956.64999999","224,824,314.69","97,871,358.04",43.53%


In [21]:
print("El segundo cuarto es el que tiene menor rentabilidad en ambos años")

El segundo cuarto es el que tiene menor rentabilidad en ambos años
